<a href="https://colab.research.google.com/github/gabrielaugustavo/LLM/blob/main/GSI073_aula0_support_vector_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

Neste notebook, um tipo de Support Vector Machine Linear.


## Preparação dos dados

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X = iris.data
y = np.where(y == 1, 1., -1.)

# 2. Preparar dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, random_state=44, stratify=y
)

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float().view(-1, 1)

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float().view(-1, 1)

A nossa Support Vector Machine é basicamente um hiperplano definido por *w* e *b* que melhor separa as classes.

In [ ]:
# 5. Definir modelo SVM linear
n_features = X_train.shape[1]
model = nn.Linear(n_features, 1)  # inclui w e b
C = 1.0
learning_rate = 0.01
epochs = 5000
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Execução do treinamento

In [ ]:
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)

    # Hinge loss
    hinge_loss = torch.clamp(1 - y_train * y_pred, min=0).mean()

    # Regularização (máxima margem)
    reg_term = 0.5 * torch.sum(model.weight ** 2)

    # Função objetivo
    loss = reg_term + C * hinge_loss

    # Backprop
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 500 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss={loss.item():.4f}")

Teste do Treinamento

In [ ]:
# Avaliação da acurácia no conjunto de teste
model.eval()  # desativa gradientes
with torch.no_grad():
    # Previsões: -1 ou +1
    y_pred_labels = torch.sign(model(X_test))

    # Comparar com y_test
    correct = (y_pred_labels.view(-1) == y_test.view(-1)).float().sum()
    accuracy = correct / y_test.shape[0]

    print(f"Acurácia no teste: {accuracy.item():.4f}")
